<a href="https://colab.research.google.com/github/bodamohannaik/DLAI-TF-DPC/blob/master/C3/W4/lyrics_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import tensorflow as tf

# Data Preparation

In [ ]:
data="In the town of Athy one Jeremy Lanigan \n Battered away til he hadnt a pound. \nHis father died and made him a man again \n Left him a farm and ten acres of ground. \nHe gave a grand party for friends and relations \nWho didnt forget him when come to the wall, \nAnd if youll but listen Ill make your eyes glisten \nOf the rows and the ructions of Lanigans Ball. \nMyself to be sure got free invitation, \nFor all the nice girls and boys I might ask, \nAnd just in a minute both friends and relations \nWere dancing round merry as bees round a cask. \nJudy ODaly, that nice little milliner, \nShe tipped me a wink for to give her a call, \nAnd I soon arrived with Peggy McGilligan \nJust in time for Lanigans Ball. \nThere were lashings of punch and wine for the ladies, \nPotatoes and cakes; there was bacon and tea, \nThere were the Nolans, Dolans, OGradys \nCourting the girls and dancing away. \nSongs they went round as plenty as water, \nThe harp that once sounded in Taras old hall,\nSweet Nelly Gray and The Rat Catchers Daughter,\nAll singing together at Lanigans Ball. \nThey were doing all kinds of nonsensical polkas \nAll round the room in a whirligig. \nJulia and I, we banished their nonsense \nAnd tipped them the twist of a reel and a jig. \nAch mavrone, how the girls got all mad at me \nDanced til youd think the ceiling would fall. \nFor I spent three weeks at Brooks Academy \nLearning new steps for Lanigans Ball. \nThree long weeks I spent up in Dublin, \nThree long weeks to learn nothing at all,\n Three long weeks I spent up in Dublin, \nLearning new steps for Lanigans Ball. \nShe stepped out and I stepped in again, \nI stepped out and she stepped in again, \nShe stepped out and I stepped in again, \nLearning new steps for Lanigans Ball. \nBoys were all merry and the girls they were hearty \nAnd danced all around in couples and groups, \nTil an accident happened, young Terrance McCarthy \nPut his right leg through miss Finnertys hoops. \nPoor creature fainted and cried Meelia murther, \nCalled for her brothers and gathered them all. \nCarmody swore that hed go no further \nTil he had satisfaction at Lanigans Ball. \nIn the midst of the row miss Kerrigan fainted, \nHer cheeks at the same time as red as a rose. \nSome of the lads declared she was painted, \nShe took a small drop too much, I suppose. \nHer sweetheart, Ned Morgan, so powerful and able, \nWhen he saw his fair colleen stretched out by the wall, \nTore the left leg from under the table \nAnd smashed all the Chaneys at Lanigans Ball. \nBoys, oh boys, twas then there were runctions. \nMyself got a lick from big Phelim McHugh. \nI soon replied to his introduction \nAnd kicked up a terrible hullabaloo. \nOld Casey, the piper, was near being strangled. \nThey squeezed up his pipes, bellows, chanters and all. \nThe girls, in their ribbons, they got all entangled \nAnd that put an end to Lanigans Ball."
print(data)

In the town of Athy one Jeremy Lanigan 
 Battered away til he hadnt a pound. 
His father died and made him a man again 
 Left him a farm and ten acres of ground. 
He gave a grand party for friends and relations 
Who didnt forget him when come to the wall, 
And if youll but listen Ill make your eyes glisten 
Of the rows and the ructions of Lanigans Ball. 
Myself to be sure got free invitation, 
For all the nice girls and boys I might ask, 
And just in a minute both friends and relations 
Were dancing round merry as bees round a cask. 
Judy ODaly, that nice little milliner, 
She tipped me a wink for to give her a call, 
And I soon arrived with Peggy McGilligan 
Just in time for Lanigans Ball. 
There were lashings of punch and wine for the ladies, 
Potatoes and cakes; there was bacon and tea, 
There were the Nolans, Dolans, OGradys 
Courting the girls and dancing away. 
Songs they went round as plenty as water, 
The harp that once sounded in Taras old hall,
Sweet Nelly Gray and The Rat Ca

In [ ]:
sentences = data.split("\n")
sentences[:5]

['In the town of Athy one Jeremy Lanigan ',
 ' Battered away til he hadnt a pound. ',
 'His father died and made him a man again ',
 ' Left him a farm and ten acres of ground. ',
 'He gave a grand party for friends and relations ']

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=None, lower=True, split=' ', oov_token="<OOV>")
# feed
tokenizer.fit_on_texts(texts=sentences)

In [ ]:
# encode to word index
sequences =  tokenizer.texts_to_sequences(texts=sentences)

In [ ]:
type(sequences), type(sequences[0])

(list, list)

In [ ]:
# prepare n length sequence to predict n+1 word
sliced_sequences = []
minimum_words = 1
for sequence in sequences:
  for i in range(minimum_words, len(sequence)):
    sliced_sequences.append(sequence[:i+1])
print(f"sequences len: {len(sequences)}")
print(f"sliced_sequences len: {len(sliced_sequences)}")

sequences len: 64
sliced_sequences len: 453


In [ ]:
print((tokenizer.sequences_to_texts(sliced_sequences[0:5])))

['in the', 'in the town', 'in the town of', 'in the town of athy', 'in the town of athy one']


In [ ]:
# pre pad the sequences
p_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences=sliced_sequences, maxlen=None, padding='pre')

In [ ]:
p_sequences.shape

(453, 11)

In [ ]:
# split  into input and output
X = p_sequences[:, :-1]
y = p_sequences[:, -1]
X.shape, y.shape

((453, 10), (453,))

In [ ]:
y = tf.keras.utils.to_categorical(y)

In [ ]:
vocab_size = len(tokenizer.word_index)+1

In [ ]:
vocab_size

264

In [ ]:
len(y[0])

264

In [ ]:
y.shape

(453, 264)

# Model

In [ ]:
model = tf.keras.Sequential(
    layers = [
              tf.keras.layers.Input(shape=X.shape[1:]),
              tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=32),
              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=64)),
              tf.keras.layers.Dense(units=vocab_size, activation='softmax')
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 32)            8448      
                                                                 
 bidirectional (Bidirectiona  (None, 128)              49664     
 l)                                                              
                                                                 
 dense (Dense)               (None, 264)               34056     
                                                                 
Total params: 92,168
Trainable params: 92,168
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=300)

Epoch 1/300
15/15 [==============================] - 4s 13ms/step - loss: 1.0379 - accuracy: 0.8057
Epoch 2/300
15/15 [==============================] - 0s 12ms/step - loss: 0.9954 - accuracy: 0.8035
Epoch 3/300
15/15 [==============================] - 0s 14ms/step - loss: 0.9838 - accuracy: 0.8234
Epoch 4/300
15/15 [==============================] - 0s 15ms/step - loss: 0.9252 - accuracy: 0.8366
Epoch 5/300
15/15 [==============================] - 0s 15ms/step - loss: 0.9130 - accuracy: 0.8521
Epoch 6/300
15/15 [==============================] - 0s 14ms/step - loss: 0.9070 - accuracy: 0.8411
Epoch 7/300
15/15 [==============================] - 0s 13ms/step - loss: 0.8623 - accuracy: 0.8543
Epoch 8/300
15/15 [==============================] - 0s 12ms/step - loss: 0.8351 - accuracy: 0.8389
Epoch 9/300
15/15 [==============================] - 0s 11ms/step - loss: 0.8072 - accuracy: 0.8631
Epoch 10/300
15/15 [==============================] - 0s 13ms/step - loss: 0.7841 - accuracy: 0.8587

In [ ]:
text = "His father died"
predict_words = 20
sequence = tokenizer.texts_to_sequences([text])[0]
print(sequence)
for i in range(20):
  current_sequence = tf.keras.preprocessing.sequence.pad_sequences([sequence], maxlen=X.shape[1], padding="pre", truncating="pre")
  pred = model.predict(current_sequence)
  current_pred_index = pred.argmax(axis=1)[0]
  if current_pred_index:
    sequence.append(current_pred_index)

[17, 75, 76]


In [ ]:
print(tokenizer.sequences_to_texts([sequence]))


['his father <OOV> him a painted a man again the glisten ask wall ground didnt boys spent ask wall when wall wall able']
